In [1]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
from multiprocessing import cpu_count
import tensorflow as tf

Using TensorFlow backend.


In [2]:
nb_classes = 20
nb_cpus = cpu_count()//2
nb_gpus = 4

image_size = (299, 299)
input_shape= (299, 299, 3)

In [3]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(1.0)(p_out)
    predictions = Dense(nb_classes, activation='softmax')(p_out)
    model = Model(inputs=base_model.input, outputs=predictions)
    
model = multi_gpu_model(model, gpus=nb_gpus)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_5 (

In [4]:
model.load_weights('weights_010_0.0698.hdf5')

In [5]:
batch_size = 128
test_path = "/home/cnn/Documents/batch6.1_bmp/train1/valid"

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(test_path, 
                                         target_size=image_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
test_img_nums = test_generator.samples
all_test_results = model.predict_generator(test_generator, 
                                           len(test_generator), 
                                           workers=nb_cpus, 
                                           use_multiprocessing=True,
                                           verbose=1)
all_labels = test_generator.classes

Found 38043 images belonging to 20 classes.
298/298 [==============================] - 470s 2s/step


In [6]:
class_label_dict = test_generator.class_indices
print(test_generator.class_indices)
def get_key(dict_, value):
    return [k for k, v in dict_.items() if v == value]

# create class num lens dict, every dict store current class predict num
total_predictions_dict = {}
for row_class_name, label in class_label_dict.items():
    total_predictions_dict[row_class_name] = {}
    for column_class_name, label in class_label_dict.items():
        total_predictions_dict[row_class_name][column_class_name] = 0

{'MC': 13, 'SCC_G': 16, 'ASCUS': 3, 'SCC_R': 17, 'LSIL_F': 12, 'AGC_B': 2, 'LSIL_E': 11, 'EC': 5, 'HSIL_M': 9, 'FUNGI': 6, 'RC': 14, 'VIRUS': 19, 'TRI': 18, 'CC': 4, 'HSIL_S': 10, 'ACTINO': 0, 'GEC': 7, 'HSIL_B': 8, 'AGC_A': 1, 'SC': 15}


In [16]:
thresh = 0.95

for i, label in enumerate(all_labels):
    predict_index = np.argmax(all_test_results[i])
    predict_det = all_test_results[i][predict_index]
    if (predict_det > thresh):
        # get the first result
        label_class_name = get_key(class_label_dict, label)[0]
        test_class_name = get_key(class_label_dict, np.argmax(all_test_results[i]))[0]
#     print(label_class_name)
#     print(total_predictions_dict[label_class_name].keys())
        total_predictions_dict[label_class_name][test_class_name] += 1

In [17]:
print(total_predictions_dict['RC'])

{'MC': 0, 'SCC_G': 0, 'ASCUS': 0, 'SCC_R': 0, 'LSIL_F': 0, 'AGC_B': 0, 'LSIL_E': 0, 'HSIL_M': 0, 'FUNGI': 0, 'RC': 444, 'VIRUS': 0, 'TRI': 0, 'AGC_A': 0, 'EC': 0, 'HSIL_S': 0, 'ACTINO': 0, 'GEC': 19, 'HSIL_B': 0, 'CC': 0, 'SC': 0}


In [18]:
import csv
out = open('confusion_matrix.csv','a', newline='')
csv_write = csv.writer(out,dialect='excel')

# write the title
line = [class_name for class_name, label in class_label_dict.items()]
line = [" "] + line + ["TOTAL"] + ["ACC"]
csv_write.writerow(line)

# write rows

true_num = 0
all_num = 0

for row_class_name, label in class_label_dict.items():
    one_class_total_predict = 0
    line = [row_class_name]
    for column_class_name, label in class_label_dict.items():
        one_class_total_predict += total_predictions_dict[row_class_name][column_class_name]
    
    for column_class_name, label in class_label_dict.items():
        one_class_cur_predict = total_predictions_dict[row_class_name][column_class_name]
        # acc
        #acc = round((one_class_cur_predict / one_class_total_predict), 4)
        #line.append(str(acc))
        # num
        line.append(one_class_cur_predict)
    print(one_class_total_predict)
    print(str(one_class_total_predict))
    line.append(str(one_class_total_predict))
    line.append(round((total_predictions_dict[row_class_name][row_class_name] / one_class_total_predict), 4))       
    print(line)
    csv_write.writerow(line)
    
    true_num += total_predictions_dict[row_class_name][row_class_name]
    all_num += one_class_total_predict
    
csv_write.writerow(["ALL_ACC"] + [round((true_num / all_num), 4)])
    
out.close()

9583
9583
['MC', 9571, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, '9583', 0.9987]
3454
3454
['SCC_G', 0, 3270, 0, 8, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 167, 0, 2, 0, 2, 0, '3454', 0.9467]
6278
6278
['ASCUS', 7, 1, 6197, 0, 59, 0, 6, 0, 7, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, '6278', 0.9871]
5483
5483
['SCC_R', 0, 6, 0, 5472, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, '5483', 0.998]
1657
1657
['LSIL_F', 0, 0, 104, 0, 1551, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '1657', 0.936]
1483
1483
['AGC_B', 0, 0, 0, 0, 0, 1441, 0, 10, 11, 0, 4, 0, 0, 0, 0, 0, 2, 8, 7, 0, '1483', 0.9717]
3921
3921
['LSIL_E', 3, 0, 41, 0, 2, 0, 3871, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, '3921', 0.9872]
743
743
['EC', 0, 0, 0, 0, 0, 2, 0, 728, 12, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, '743', 0.9798]
6365
6365
['HSIL_M', 0, 3, 0, 0, 2, 17, 0, 0, 6315, 0, 0, 0, 0, 0, 19, 0, 3, 6, 0, 0, '6365', 0.9921]
5806
5806
['FUNGI', 0, 2, 0, 0, 0, 0, 0, 0, 0, 5804, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '5806', 0.9997]
463
463
